# 01 — EDA (Polish Companies Bankruptcy) — Part 2

**Goal (Steps 6–9):** Turn the audit into concrete cleaning decisions and diagnostics:
- Decide on missingness handling (drop high-NA features; impute others; add missing indicators when useful)
- Winsorize outliers per horizon (robustify heavy-tailed ratios)
- Diagnose multicollinearity with correlations & VIF
- Prepare a *candidate* cleaned feature set and save artifacts

> Run this notebook from your repo root so the relative `data/` paths resolve.
> If you prefer to continue inside your Part 1 notebook, you can copy-paste the cells instead.


### Step 6 — Imports & constants

**Why:** Load required packages and set deterministic options. We'll reuse the ARFF loader from Part 1 to re-create `df_all` locally here.

In [1]:
# If any imports fail, install them in your environment:
# uv pip install scipy statsmodels pyarrow

from pathlib import Path
import warnings
import numpy as np
import pandas as pd
from scipy.io import arff
from collections import defaultdict

from statsmodels.stats.outliers_influence import variance_inflation_factor

pd.set_option("display.max_rows", 200)
pd.set_option("display.max_columns", 200)
pd.set_option("display.width", 180)
warnings.filterwarnings("ignore")
RANDOM_STATE = 42

REPO_ROOT = Path.cwd()
DATA_DIR = REPO_ROOT / "data" / "polish-companies-bankruptcy" / "polish+companies+bankruptcy+data"
OUT_DIR = REPO_ROOT / "data" / "processed"
OUT_DIR.mkdir(parents=True, exist_ok=True)

print('✅ Imports OK')

✅ Imports OK


*(No interpretation needed here.)*

### Step 7 — Reload all horizons into a single DataFrame

**Why:** Keep this notebook self-contained. We recreate `df_all` with a `y` target and `horizon` column.

In [2]:
def load_arff_to_df(path: Path, horizon: int) -> pd.DataFrame:
    data, meta = arff.loadarff(str(path))
    df = pd.DataFrame(data)
    # Decode byte columns that represent strings
    for c in df.columns:
        if df[c].dtype == object:
            try:
                df[c] = df[c].str.decode('utf-8')
            except Exception:
                pass

    # Identify target column
    target_col_candidates = [c for c in df.columns if c.lower() in {'class', 'target', 'bankrupt'}]
    if not target_col_candidates:
        raise ValueError(f"No obvious target column found in {path.name}. Columns: {list(df.columns)[:10]}...")
    target_col = target_col_candidates[0]

    # Normalize target to {0,1}
    mapping = {'0':0,'1':1,'N':0,'B':1,'No':0,'Yes':1,'negative':0,'positive':1,'bankrupt':1,'non-bankrupt':0,'false':0,'true':1,0:0,1:1}
    def map_target(v):
        if isinstance(v, bytes):
            v = v.decode('utf-8', errors='ignore')
        return mapping.get(v, v)
    df[target_col] = df[target_col].map(map_target)
    if not pd.api.types.is_numeric_dtype(df[target_col]):
        try:
            df[target_col] = df[target_col].astype(int)
        except Exception:
            df[target_col] = pd.factorize(df[target_col])[0]
            if df[target_col].value_counts().shape[0] == 2:
                counts = df[target_col].value_counts()
                minority = counts.idxmin()
                df[target_col] = (df[target_col] == minority).astype(int)

    df['horizon'] = horizon
    if target_col != 'y':
        df = df.rename(columns={target_col: 'y'})
    return df

dfs = [load_arff_to_df(DATA_DIR / f"{h}year.arff", h) for h in [1,2,3,4,5]]
df_all = pd.concat(dfs, axis=0, ignore_index=True)

feature_cols = [c for c in df_all.columns if c not in ['y', 'horizon']]
print('df_all shape:', df_all.shape, '| features:', len(feature_cols))

df_all shape: (43405, 66) | features: 64


**Interpretation (after running):**  
- Confirm the shape and that you have 64 feature columns plus `y` and `horizon` (total 66).

### Step 8 — Missingness decisions and imputation plan

**Why:** Attr37 has ~44% missing; a few others are 5–13%. We will:
1. **Drop** features with missingness > 35% (to avoid noise/leakage from imputation).
2. **Add missingness indicators** for features with NA ≥ 5% (to capture “missing is informative”).
3. **Impute** remaining NAs with **median per horizon** (robust to outliers, respects horizon differences).

In [3]:
# Compute missingness per feature
na_pct = df_all[feature_cols].isna().mean().sort_values(ascending=False)
high_na_drop = na_pct[na_pct > 0.35].index.tolist()  # e.g., Attr37
indicator_feats = na_pct[(na_pct >= 0.05) & (na_pct <= 0.35)].index.tolist()

print('High-NA features to DROP (>35% NA):', high_na_drop)
print('Features to add as MISSING INDICATORS (>=5% NA):', indicator_feats[:20], '... total', len(indicator_feats))

# Build working dataframe
Xy = df_all.copy()
for col in indicator_feats:
    Xy[f"{col}__isna"] = Xy[col].isna().astype(int)

# Impute per horizon by median
for h in sorted(Xy['horizon'].unique()):
    mask = Xy['horizon'] == h
    med = Xy.loc[mask, feature_cols].median(numeric_only=True)
    Xy.loc[mask, feature_cols] = Xy.loc[mask, feature_cols].fillna(med)

# Drop high-NA features
Xy = Xy.drop(columns=high_na_drop)

print('After indicators+impute+drop → shape:', Xy.shape)

High-NA features to DROP (>35% NA): ['Attr37']
Features to add as MISSING INDICATORS (>=5% NA): ['Attr21', 'Attr27'] ... total 2
After indicators+impute+drop → shape: (43405, 67)


**Interpretation (after running):**  
- List which columns were dropped (likely `Attr37`).  
- Note how many indicator variables were added.  
- Confirm the new shape and that `y` still has its original positive rate.

### Step 9 — Outliers: horizon-wise winsorization at 1% / 99%

**Why:** Financial ratios are heavy-tailed. Winsorization reduces the impact of extreme values that could destabilize Logit and distort distances.

**Plan:** For each horizon, clip each numeric feature at its 1st and 99th percentiles (computed within that horizon).

In [4]:
def winsorize_by_horizon(df: pd.DataFrame, num_cols, lower=0.01, upper=0.99, group_col='horizon'):
    df_w = df.copy()
    for h, sub in df.groupby(group_col):
        q_low = sub[num_cols].quantile(lower)
        q_hi  = sub[num_cols].quantile(upper)
        idx = df[group_col] == h
        df_w.loc[idx, num_cols] = np.clip(df.loc[idx, num_cols], q_low, q_hi, axis=1)
    return df_w

num_cols = [c for c in Xy.columns if c not in ['y', 'horizon'] and (pd.api.types.is_numeric_dtype(Xy[c]))]
Xw = winsorize_by_horizon(Xy, num_cols=num_cols, lower=0.01, upper=0.99, group_col='horizon')

print('Winsorized numeric columns:', len(num_cols))
print('Example quantiles (pre vs post) for a few columns:')
for c in num_cols[:5]:
    pre = Xy[c].quantile([0.0,0.01,0.5,0.99,1.0]).to_list()
    post = Xw[c].quantile([0.0,0.01,0.5,0.99,1.0]).to_list()
    print(f"  {c}: pre {pre} → post {post}")

Winsorized numeric columns: 65
Example quantiles (pre vs post) for a few columns:
  Attr1: pre [-463.89, -0.5767864, 0.049658, 0.6059115999999999, 94.28] → post [-0.6788284, -0.5627992, 0.049658, 0.5602750000000001, 0.6683501999999996]
  Attr2: pre [-430.87, 0.01852464, 0.4719, 2.0562319999999983, 480.96] → post [0.01542695, 0.02091588, 0.4719, 1.9171080000000107, 2.478766000000001]
  Attr3: pre [-479.96, -1.000576, 0.19661, 0.8897984, 28.336] → post [-1.286353, -0.9036896000000001, 0.19661, 0.8869011999999998, 0.9166460000000005]
  Attr4: pre [-0.40311, 0.2001008, 1.5711, 32.30139999999999, 53433.0] → post [0.1466768, 0.2310428, 1.5711, 27.28111999999998, 44.04940000000004]
  Attr5: pre [-11903000.0, -2969.528, -0.95364, 2873.8439999999855, 1250100.0] → post [-3458.5980000000004, -2807.7000000000003, -0.95364, 2228.651999999997, 3948.816000000001]


**Interpretation (after running):**  
- Check that the extreme min/max values are now truncated while medians remain very similar.  
- If any feature behaves strangely (e.g., almost all values clipped), flag it for review/removal.

### Step 10 — Collinearity diagnostics (correlations & VIF)

**Why:** Many ratios are algebraically related → unstable coefficients in Logit. We'll:
- Identify highly correlated pairs (|r| ≥ 0.9)
- Compute **VIF** and flag features with VIF > 10
- Propose a reduced set by greedily removing one feature from each high-corr pair (keep features with fewer missingness / lower VIF)

In [5]:
# Use the winsorized, imputed features
Z = Xw.copy()
feat_cols = [c for c in Z.columns if c not in ['y','horizon'] and not c.endswith('__isna')]

# Correlation matrix on a sample (for speed if needed)
corr = Z[feat_cols].corr().abs()

# List highly correlated pairs (upper triangle)
high_pairs = []
cols = corr.columns
for i in range(len(cols)):
    for j in range(i+1, len(cols)):
        r = corr.iloc[i, j]
        if r >= 0.9:
            high_pairs.append((cols[i], cols[j], float(r)))
high_pairs_sorted = sorted(high_pairs, key=lambda x: -x[2])
print('Highly correlated pairs (|r| ≥ 0.9):', len(high_pairs_sorted))
print(high_pairs_sorted[:15])

# VIF (can be slow; run on a subset if needed)
import numpy as np
from statsmodels.tools.tools import add_constant

X_for_vif = Z[feat_cols].replace([np.inf, -np.inf], np.nan).fillna(0.0)
X_for_vif = add_constant(X_for_vif, has_constant='add')
vif_values = []
for k, col in enumerate(X_for_vif.columns):
    if col == 'const':
        continue
    vif_values.append((col, variance_inflation_factor(X_for_vif.values, k)))

vif_df = pd.DataFrame(vif_values, columns=['feature','VIF']).sort_values('VIF', ascending=False)
print('\nTop VIFs:')
print(vif_df.head(20))

# Greedy reduction from high-corr pairs
to_drop = set()
kept = set(feat_cols)
for a,b,r in high_pairs_sorted:
    if a in to_drop or b in to_drop:
        continue
    # heuristic: drop the one with higher VIF (if available), else drop second
    va = float(vif_df.loc[vif_df['feature']==a, 'VIF'].values[0]) if (vif_df['feature']==a).any() else 1.0
    vb = float(vif_df.loc[vif_df['feature']==b, 'VIF'].values[0]) if (vif_df['feature']==b).any() else 1.0
    drop = a if va >= vb else b
    to_drop.add(drop)
    kept.discard(drop)

reduced_features = sorted(list(kept))
print('\nProposed reduced feature count:', len(reduced_features), ' (dropped', len(to_drop), 'from high-corr pairs)')

Highly correlated pairs (|r| ≥ 0.9): 31
[('Attr7', 'Attr14', 0.9999791826597377), ('Attr14', 'Attr18', 0.9995744702753196), ('Attr7', 'Attr18', 0.9993786248329541), ('Attr8', 'Attr17', 0.9971714299955167), ('Attr16', 'Attr26', 0.9939401178067024), ('Attr2', 'Attr10', 0.9879503478873164), ('Attr19', 'Attr23', 0.9865518614198378), ('Attr1', 'Attr7', 0.9824585694091564), ('Attr1', 'Attr14', 0.9824414012994532), ('Attr1', 'Attr18', 0.9818561001579155), ('Attr28', 'Attr54', 0.9772791616999396), ('Attr53', 'Attr54', 0.9761483741374839), ('Attr7', 'Attr11', 0.9738666392999702), ('Attr11', 'Attr14', 0.9738394074905476), ('Attr11', 'Attr18', 0.9732412649272559)]

Top VIFs:
   feature            VIF
13  Attr14  266359.514765
6    Attr7  183398.302804
17  Attr18    8875.954977
7    Attr8     304.159524
16  Attr17     299.461469
15  Attr16     141.194349
25  Attr26     131.652272
18  Attr19     128.326955
9   Attr10     100.072486
1    Attr2      96.743628
22  Attr23      79.181790
0    Attr1     

**Interpretation (after running):**  
- Note how many ≥0.9 correlation pairs exist; list any obviously redundant groups.  
- Inspect top VIFs — features with VIF > 10 are concerning for Logit inference (not a problem for RF).  
- The greedy drop list is a *proposal*; we’ll validate that predictive performance remains stable after reduction.

### Step 11 — Save cleaned artifacts

**Why:** Keep a consistent cleaned dataset for modeling notebooks.

In [6]:
# Save winsorized, imputed full set
full_path = OUT_DIR / 'poland_clean_full.parquet'
Z.to_parquet(full_path, index=False)

# Also save a reduced-features view
reduced_path = OUT_DIR / 'poland_clean_reduced.parquet'
cols_out = ['y','horizon'] + reduced_features + [c for c in Z.columns if c.endswith('__isna')]
Z[cols_out].to_parquet(reduced_path, index=False)

print('Saved:', full_path.name, 'and', reduced_path.name, 'in', OUT_DIR)

Saved: poland_clean_full.parquet and poland_clean_reduced.parquet in /Users/reebal/FH-Wedel/WS25/seminar-bankruptcy-prediction/data/processed


**Interpretation (after running):**  
- Confirm files were written to `data/processed/`.  
- We’ll use `poland_clean_full.parquet` for robust baselines and compare to the `reduced` variant for Logit stability.